In [1]:
import random

import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn 
import torch.nn.functional as F
import copy 
import math 
import numpy as np
import pandas as pd

# 定义字典
words_x = '<PAD>,1,2,3,4,5,6,7,8,9,0,<SOS>,<EOS>,+'
vocab_x = {word: i for i, word in enumerate(words_x.split(','))}
vocab_xr = [k for k, v in vocab_x.items()] #反查词典

words_y = '<PAD>,1,2,3,4,5,6,7,8,9,0,<SOS>,<EOS>'
vocab_y = {word: i for i, word in enumerate(words_y.split(','))}
vocab_yr = [k for k, v in vocab_y.items()] #反查词典


#两数相加数据集
def get_data():
    # 定义词集合
    words = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

    # 每个词被选中的概率
    p = np.array([7, 5, 5, 7, 6, 5, 7, 6, 5, 7])
    p = p / p.sum()

    # 随机采样n1个词作为s1
    n1 = random.randint(10, 20)
    s1 = np.random.choice(words, size=n1, replace=True, p=p)
    s1 = s1.tolist()

    # 随机采样n2个词作为s2
    n2 = random.randint(10, 20)
    s2 = np.random.choice(words, size=n2, replace=True, p=p)
    s2 = s2.tolist()

    # x等于s1和s2字符上的相加
    x = s1 + ['+'] + s2
    
    # y等于s1和s2数值上的相加
    y = int(''.join(s1)) + int(''.join(s2))
    y = list(str(y))
    
    # 加上首尾符号
    x = ['<SOS>'] + x + ['<EOS>']
    y = ['<SOS>'] + y + ['<EOS>']

    # 补pad到固定长度
    x = x + ['<PAD>'] * 50
    y = y + ['<PAD>'] * 51
    x = x[:50]
    y = y[:51]

    # 编码成token
    token_x = [vocab_x[i] for i in x]
    token_y = [vocab_y[i] for i in y]

    # 转tensor
    tensor_x = torch.LongTensor(token_x)
    tensor_y = torch.LongTensor(token_y)
    return tensor_x, tensor_y


def show_data(tensor_x,tensor_y) ->"str":
    words_x = "".join([vocab_xr[i] for i in tensor_x.tolist()])
    words_y = "".join([vocab_yr[i] for i in tensor_y.tolist()])
    return words_x,words_y


x,y = get_data() 
print(x,y,"\n")
print(show_data(x,y))

tensor([11,  4,  8,  4,  6,  8, 10,  9,  1,  2, 10,  4,  9,  7,  9, 13,  6,  1,
         3,  6,  7,  1,  7, 10,  5, 10,  5,  5,  9,  6, 12,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) tensor([11,  1, 10,  9,  8,  3,  5,  2,  6,  1,  7,  1, 10,  5,  7,  5, 12,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) 

('<SOS>48468091204979+61367170505596<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>', '<SOS>109835261710575<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>')


In [2]:
vocab_y

{'<PAD>': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '0': 10,
 '<SOS>': 11,
 '<EOS>': 12}

In [3]:
# 定义数据集
class TwoSumDataset(torch.utils.data.Dataset):
    def __init__(self,size = 100000):
        super(Dataset, self).__init__()
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, i):
        return get_data()
    
ds_train = TwoSumDataset(size = 100000)
ds_val = TwoSumDataset(size = 10000)


# 数据加载器
dl_train = DataLoader(dataset=ds_train,
         batch_size=200,
         drop_last=True,
         shuffle=True)

dl_val = DataLoader(dataset=ds_val,
         batch_size=200,
         drop_last=True,
         shuffle=False)


for src,tgt in dl_train:
    print(src.shape)
    print(tgt.shape)
    break

torch.Size([200, 50])
torch.Size([200, 51])


In [ ]:


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class ScaledDotProductAttention(nn.Module):
    "Compute 'Scaled Dot Product Attention'"
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self,query, key, value, mask=None, dropout=None):
        d_k = query.size(-1)
        scores = query@key.transpose(-2,-1) / math.sqrt(d_k)     
        print("q,k,v shape",query.shape,key.shape,value.shape)
        if mask is not None:
            print("scores shape,mask shape",scores.shape,mask.shape)
            scores = scores.masked_fill(mask == 0, -1e20)
        p_attn = F.softmax(scores, dim = -1)
        print("p_attn.shape",p_attn.shape)
        if dropout is not None:
            p_attn = dropout(p_attn)
        return p_attn@value, p_attn

class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadAttention, self).__init__()
        assert d_model % h == 0 # 保证h能整除输入tensor的向量维度
        # We assume d_v always equals d_k
        self.d_k = d_model // h # 每个头的向量维度
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None #记录 attention矩阵结果
        self.dropout = nn.Dropout(p=dropout)
        self.attention = ScaledDotProductAttention()
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query =  self.linears[0](query).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        key =  self.linears[1](key).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        value  =  self.linears[2](value).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        # query, key, value = [
        #     l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
        #      for l, x in zip(self.linears, (query, key, value))
        # ]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = self.attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)


#为了让训练过程与解码过程信息流一致，遮挡tgt序列后面元素，设置其注意力为0
def tril_mask(data):
    "Mask out future positions."
    size = data.size(-1) #size为序列长度
    full = torch.full((1,size,size),1,dtype=torch.int,device=data.device)
    mask = torch.tril(full).bool() 
    return mask


#设置对<PAD>的注意力为0
def pad_mask(data, pad=0):
    "Mask out pad positions."
    mask = (data!=pad).unsqueeze(-2)
    return mask 


#计算一个batch数据的src_mask和tgt_mask
class MaskedBatch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, tgt=None, pad=0):
        self. = 
        self.src_mask = pad_mask(,pad)
        if tgt is not None:
            self.tgt = tgt[:,:-1] #训练时,拿tgt的每一个词输入,去预测下一个词,所以最后一个词无需输入
            self.tgt_y = tgt[:, 1:] #第一个总是<SOS>无需预测，预测从第二个词开始
            self.tgt_mask = \
                self.make_tgt_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y!= pad).sum() 
    
    @staticmethod
    def make_tgt_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_pad_mask = pad_mask(tgt,pad)
        tgt_tril_mask = tril_mask(tgt)
        tgt_mask = tgt_pad_mask & (tgt_tril_mask)
        return tgt_mask
    

import plotly.express as px 
# 测试tril_mask 
mask = tril_mask(torch.zeros(1,10)) #序列长度为10
#sns.heatmap(mask[0],cmap=sns.cm.rocket);
px.imshow(mask[0],color_continuous_scale="blues",height=600,width=600)

In [5]:
#测试 ScaledDotProductAttention

query = torch.tensor([[[0.0,1.414],[1.414,0.0],[1.0,1.0],[-1.0,1.0],[1.0,-1.0]]])
print(query.shape)
key = query.clone() 
value = query.clone()

attention = ScaledDotProductAttention()

#没有mask 
out,p_att = attention(query, key, value)
fig = px.imshow(p_att[0],color_continuous_scale="blues",
                title="without mask",height=600,width=600)
fig.show()


torch.Size([1, 5, 2])
q,k,v shape torch.Size([1, 5, 2]) torch.Size([1, 5, 2]) torch.Size([1, 5, 2])
p_attn.shape torch.Size([1, 5, 5])


In [6]:
tril_mask(torch.zeros(3,5))==0,query.shape,key.shape, value.shape

(tensor([[[False,  True,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False, False,  True],
          [False, False, False, False, False]]]),
 torch.Size([1, 5, 2]),
 torch.Size([1, 5, 2]),
 torch.Size([1, 5, 2]))

In [7]:
#考虑mask
out,p_att = attention(query, key, value, mask = tril_mask(torch.zeros(3,5)))
fig = px.imshow(p_att[0],color_continuous_scale="blues",
                height=600,width=600,
                title="with mask")
fig.show()

q,k,v shape torch.Size([1, 5, 2]) torch.Size([1, 5, 2]) torch.Size([1, 5, 2])
scores shape,mask shape torch.Size([1, 5, 5]) torch.Size([1, 5, 5])
p_attn.shape torch.Size([1, 5, 5])


In [8]:
cross_attn = MultiHeadAttention(h=2, d_model=4)
cross_attn.eval()
q1 = torch.tensor([[[0.1,0.1,0.1,0.1],[0.1,0.3,0.1,0.3],[0.1,0.3,0.1,0.3]]])
print("q1",q1.unsqueeze(1))
print(q1.shape)
k1 = q1.clone()
v1 = q1.clone()
tgt_mask = tril_mask(torch.zeros(3,3))
# tgt_mask=None

out1 = cross_attn.forward(q1,k1,v1,mask = tgt_mask)
print("out1:\n",out1,out1.shape)

# 改变序列的第2个元素取值，由于有mask的遮挡，不会影响第1个输出
q2 = torch.tensor([[[0.1,0.1,0.1,0.1],[0.4,0.5,0.5,0.8]]])
k2 = q2.clone()
v2 = q2.clone()
tgt_mask = tril_mask(torch.zeros(2,2))
out2 = cross_attn.forward(q2,k2,v2,mask = tgt_mask)
print("out2:\n",out2,out2.shape)


q1 tensor([[[[0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.3000, 0.1000, 0.3000],
          [0.1000, 0.3000, 0.1000, 0.3000]]]])
torch.Size([1, 3, 4])
q,k,v shape torch.Size([1, 2, 3, 2]) torch.Size([1, 2, 3, 2]) torch.Size([1, 2, 3, 2])
scores shape,mask shape torch.Size([1, 2, 3, 3]) torch.Size([1, 1, 3, 3])
p_attn.shape torch.Size([1, 2, 3, 3])
out1:
 tensor([[[ 0.0982, -0.5530,  0.2645, -0.0386],
         [ 0.0876, -0.5403,  0.2525, -0.0266],
         [ 0.0840, -0.5361,  0.2484, -0.0226]]], grad_fn=<ViewBackward0>) torch.Size([1, 3, 4])
q,k,v shape torch.Size([1, 2, 2, 2]) torch.Size([1, 2, 2, 2]) torch.Size([1, 2, 2, 2])
scores shape,mask shape torch.Size([1, 2, 2, 2]) torch.Size([1, 1, 2, 2])
p_attn.shape torch.Size([1, 2, 2, 2])
out2:
 tensor([[[ 0.0982, -0.5530,  0.2645, -0.0386],
         [ 0.1080, -0.4897,  0.2482, -0.0342]]], grad_fn=<ViewBackward0>) torch.Size([1, 2, 4])


In [ ]:
# 测试MaskedBatch
mbatch = MaskedBatch(src = src,tgt = tgt, pad = 0)
print(mbatch.src.shape)
print(mbatch.tgt.shape)
print(mbatch.tgt_y.shape)

print(mbatch.src_mask.shape)
print(mbatch.tgt_mask.shape)
px.imshow(mbatch.tgt_mask[0],color_continuous_scale="blues",width=600,height=600)


torch.Size([200, 50])
torch.Size([200, 50])
torch.Size([200, 50])
torch.Size([200, 1, 50])
torch.Size([200, 50, 50])


In [10]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)  #线性层默认作用在最后一维度
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.linear2(self.dropout(F.relu(self.linear1(x))))
class LayerNorm(nn.Module):
    "Construct a layernorm module (similar to torch.nn.LayerNorm)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.weight * (x - mean) / (std + self.eps) + self.bias

class ResConnection(nn.Module):
    """
    A residual connection with a layer norm.
    Note the norm is at last according to the paper, but it may be better at first.
    """
    def __init__(self, size, dropout, norm_first=True):
        super(ResConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        self.norm_first = norm_first

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        if self.norm_first:
            return x + self.dropout(sublayer(self.norm(x)))
        else:
            return self.norm(x + self.dropout(sublayer(x)))
# 单词嵌入
class WordEmbedding(nn.Module):
    def __init__(self, d_model, vocab):
        super(WordEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model) #note here, multiply sqrt(d_model)


In [11]:
# 位置编码
class PositionEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)
    

pe = PositionEncoding(120, 0)
z = pe.forward(torch.zeros(1, 100, 120))


print("z shape",z.shape)
df = pd.DataFrame(z[0, :, [0,20,60,110]].data.numpy(),columns = ["dim"+c for c in ['0','20','60','110']])
df.insert(0,"x",np.arange(100))
px.line(df, x = "x",y = ["dim"+c for c in ['0','20','60','110']]).show()
# 不同纬度在序列位置上的编码值分布

z shape torch.Size([1, 100, 120])


In [12]:
df.head()

,x,dim0,dim20,dim60,dim110
0,0,0.000000,0.000000,0.000000,0.000000
1,1,0.841471,0.213781,0.010000,0.000215
2,2,0.909297,0.417677,0.019999,0.000431
3,3,0.141120,0.602261,0.029995,0.000646
4,4,-0.756802,0.758999,0.039989,0.000862


In [13]:
torch.arange(0, 5000).unsqueeze(1).shape,torch.exp(torch.arange(0, 4, 2) * -(math.log(10000.0) / 4)).shape

(torch.Size([5000, 1]), torch.Size([2]))

In [14]:
max_len = 5000
d_model = 512
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len).unsqueeze(1)
print("position shape",position.shape)
div_term = torch.exp(torch.arange(0, d_model, 2) *
                        -(math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
pe = pe.unsqueeze(0)
pe.shape

position shape torch.Size([5000, 1])


torch.Size([1, 5000, 512])

In [15]:
torch.arange(0, d_model, 2).shape

torch.Size([256])

In [16]:
px.imshow(np.squeeze(z.data.numpy()) ,color_continuous_scale="blues",width=1000,height=800)

In [ ]:
class TransformerEncoderLayer(nn.Module):
    "TransformerEncoderLayer is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.res_layers = clones(ResConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.res_layers[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.res_layers[1](x, self.feed_forward)
    
    
class TransformerEncoder(nn.Module):
    "TransformerEncoder is a stack of N TransformerEncoderLayer"
    def __init__(self, layer, N):
        super(TransformerEncoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
    @classmethod
    def from_config(cls,N=6,d_model=512, d_ff=2048, h=8, dropout=0.1):
        attn = MultiHeadAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        layer = TransformerEncoderLayer(d_model, attn, ff, dropout)
        return cls(layer,N)
    
from torchkeras import summary 

src_embed = nn.Sequential(WordEmbedding(d_model=32, vocab = len(vocab_x)), 
                          PositionEncoding(d_model=32, dropout=0.1))
encoder = TransformerEncoder.from_config(N=3,d_model=32, d_ff=128, h=8, dropout=0.1)
src_mask = pad_mask()
memory = encoder(*[src_embed(),src_mask]) 
summary(encoder,input_data_args = [src_embed(),src_mask]);


No module named 'sklearn'


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torc

In [18]:
class TransformerDecoderLayer(nn.Module):
    "TransformerDecoderLayer is made of self-attn, cross-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, cross_attn, feed_forward, dropout):
        super(TransformerDecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.cross_attn = cross_attn
        self.feed_forward = feed_forward
        self.res_layers = clones(ResConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.res_layers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.res_layers[1](x, lambda x: self.cross_attn(x, m, m, src_mask))
        return self.res_layers[2](x, self.feed_forward)
    

class TransformerDecoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(TransformerDecoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
    @classmethod
    def from_config(cls,N=6,d_model=512, d_ff=2048, h=8, dropout=0.1):
        self_attn = MultiHeadAttention(h, d_model)
        cross_attn = MultiHeadAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        layer = TransformerDecoderLayer(d_model, self_attn, cross_attn, ff, dropout)
        return cls(layer,N)


from torchkeras import summary 

mbatch = MaskedBatch(src=src,tgt=tgt,pad=0)

src_embed = nn.Sequential(WordEmbedding(d_model=32, vocab = len(vocab_x)), 
                          PositionEncoding(d_model=32, dropout=0.1))
encoder = TransformerEncoder.from_config(N=3,d_model=32, d_ff=128, h=8, dropout=0.1)
memory = encoder(src_embed(src),mbatch.src_mask) 

tgt_embed = nn.Sequential(WordEmbedding(d_model=32, vocab = len(vocab_y)), 
                          PositionEncoding(d_model=32, dropout=0.1))
decoder = TransformerDecoder.from_config(N=3,d_model=32, d_ff=128, h=8, dropout=0.1)

result = decoder.forward(tgt_embed(mbatch.tgt),memory,mbatch.src_mask,mbatch.tgt_mask) 
summary(decoder,input_data_args = [tgt_embed(mbatch.tgt),memory,
                              mbatch.src_mask,mbatch.tgt_mask]);


decoder.eval() 
mbatch.tgt[0][1]=8
result = decoder.forward(tgt_embed(mbatch.tgt),memory,mbatch.src_mask,mbatch.tgt_mask)
print(torch.sum(result[0][0])) 

mbatch.tgt[0][1]=7
result = decoder.forward(tgt_embed(mbatch.tgt),memory,mbatch.src_mask,mbatch.tgt_mask)
print(torch.sum(result[0][0]))

q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 50, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape tor

In [19]:

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)


generator = Generator(d_model = 32, vocab = len(vocab_y)) 
log_probs  = generator(result)
probs = torch.exp(log_probs)
print("output_probs.shape:",probs.shape)
print("sum(probs)=1:") 
print(torch.sum(probs,dim = -1)[0]) 

summary(generator,input_data = result);

output_probs.shape: torch.Size([200, 50, 13])
sum(probs)=1:
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SelectBackward0>)
--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Linear-1                                [-1, 50, 13]                  429
Total params: 429
Trainable params: 429
Non-trainable params: 0
--------------------------------------------------------------------------
Input size (MB): 0.000076
Forward/backward pass size (MB): 0.004959
Params size (MB): 0.001637
Estimat

In [20]:

from torch import nn 
class Transformer(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        self.reset_parameters()
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.generator(self.decode(self.encode(src, src_mask), 
                src_mask, tgt, tgt_mask))
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
    @classmethod
    def from_config(cls,src_vocab,tgt_vocab,N=6,d_model=512, d_ff=2048, h=8, dropout=0.1):
        encoder = TransformerEncoder.from_config(N=N,d_model=d_model,
                  d_ff=d_ff, h=h, dropout=dropout)
        decoder = TransformerDecoder.from_config(N=N,d_model=d_model,
                  d_ff=d_ff, h=h, dropout=dropout)
        src_embed = nn.Sequential(WordEmbedding(d_model, src_vocab), PositionEncoding(d_model, dropout))
        tgt_embed = nn.Sequential(WordEmbedding(d_model, tgt_vocab), PositionEncoding(d_model, dropout))
        
        generator = Generator(d_model, tgt_vocab)
        return cls(encoder, decoder, src_embed, tgt_embed, generator)
    
    def reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    


from torchkeras import summary 
net = Transformer.from_config(src_vocab = len(vocab_x),tgt_vocab = len(vocab_y),
                   N=2, d_model=32, d_ff=128, h=8, dropout=0.1)

mbatch = MaskedBatch(src=src,tgt=tgt,pad=0)

summary(net,input_data_args = [mbatch.src,mbatch.tgt,mbatch.src_mask,mbatch.tgt_mask]);

q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 50, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape torch.Size([200, 8, 50, 50]) torch.Size([200, 1, 1, 50])
p_attn.shape torch.Size([200, 8, 50, 50])
q,k,v shape torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4]) torch.Size([200, 8, 50, 4])
scores shape,mask shape tor